<a href="https://colab.research.google.com/github/haimanhamzah/ML-Classification/blob/main/Classification_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline, make_pipeline

# Storing the training set in a dataframe
all_df = pd.read_excel("TrainDataset2023.xls", index_col=False)

# Dropping the ID column as it is irrelevant to training
all_df = all_df.drop(['ID'], axis=1)
all_df.head()

,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,1,144.0,41.0,0,0,0,1,3,3,1,...,0.517172,0.375126,3.325332,0.002314,3880771.500,473.464852,0.000768,0.182615,0.030508,0.000758
1,0,142.0,39.0,1,1,0,0,3,3,1,...,0.444391,0.444391,3.032144,0.005612,2372009.744,59.459710,0.004383,0.032012,0.001006,0.003685
2,1,135.0,31.0,0,0,0,1,2,1,1,...,0.534549,0.534549,2.485848,0.006752,1540027.421,33.935384,0.007584,0.024062,0.000529,0.006447
3,0,12.0,35.0,0,0,0,1,3,3,1,...,0.506185,0.506185,2.606255,0.003755,6936740.794,46.859265,0.005424,0.013707,0.000178,0.004543
4,0,109.0,61.0,1,0,0,0,2,1,1,...,0.462282,0.462282,2.809279,0.006521,1265399.054,39.621023,0.006585,0.034148,0.001083,0.005626


In [ ]:
# Data inspection
# We can see below that the missing values are messing with the max values of certain features
# It's also less obvious but there are outliers
all_df.info()
all_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 119 entries, pCR (outcome) to original_ngtdm_Strength
dtypes: float64(108), int64(11)
memory usage: 372.0 KB


,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.00000,...,4.000000e+02,4.000000e+02,4.000000e+02,400.000000,4.000000e+02,400.000000,400.000000,400.000000,400.000000,400.000000
mean,12.697500,56.000208,51.804674,0.547500,2.902500,2.797500,2.830000,9.875000,6.562500,8.63250,...,3.957637e-01,3.911005e-01,2.722189e+00,0.003347,5.679717e+07,178.311246,32500.032620,0.056935,0.005965,0.029322
std,111.107417,27.137584,10.948522,0.498362,49.932114,49.937068,49.935558,86.092911,70.444284,86.20034,...,1.666319e-01,1.615922e-01,7.648849e-01,0.002419,7.063846e+08,1045.453432,177545.921568,0.047179,0.008379,0.115915
min,0.000000,0.000000,23.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.00000,...,7.050000e-11,7.050000e-11,-3.200000e-16,0.000008,0.000000e+00,0.000000,0.000248,0.000000,0.000000,0.000000
25%,0.000000,38.000000,44.516769,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.00000,...,3.199017e-01,3.184398e-01,2.340783e+00,0.001389,1.030473e+06,18.760570,0.001826,0.018628,0.000310,0.001464
50%,0.000000,55.000000,51.019507,1.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.00000,...,4.095627e-01,4.054695e-01,2.814884e+00,0.002944,3.277334e+06,67.929659,0.004383,0.047740,0.002330,0.003276
75%,0.000000,73.000000,60.000000,1.000000,1.000000,1.000000,1.000000,3.000000,2.000000,1.00000,...,5.000049e-01,4.956920e-01,3.304411e+00,0.004798,9.079686e+06,157.370294,0.013769,0.085321,0.007962,0.009479
max,999.000000,144.000000,79.603012,1.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.00000,...,8.773779e-01,8.571429e-01,4.947427e+00,0.011301,1.390001e+10,20764.693790,1000000.000000,0.285100,0.060742,1.145601


In [ ]:
# DATA PRE-PROCESSING
# dropping rows that have missing pCR values
# since pCR is a classification target, a missing value means its effectively useless for our purposes

j = 0
all_df_new = all_df
for i in all_df_new['pCR (outcome)'] :
  if i == 999 :
    all_df_new = all_df_new.drop(axis=0, index=j)
  j += 1

In [ ]:
# Undersampling funciton
# Removes a number of entries (roughly) equal to the number of zeroes minus the number of ones
# This balances the dataset
def undersampler(df):
  import random
  import math

  # The seed for which random zeroes are removed
  seed = 10
  num_instances = len(df.index)

  # Resets the in-built pandas index column to prevent removal errors
  df.index = [x for x in range(num_instances)]

  # initialising variables
  df_target = df['pCR (outcome)']

  zeroes_index_list = []
  return_index_list = []
  row_list = []
  num_ones = 0;

  # Setting seed
  random.seed(seed)

  # Counting the number of ones and zeroes in the dataset
  # Storing the indexes of the zeroes in a list for later use
  j = 0
  for val in df_target :
    if(val == 1) :
      num_ones = num_ones + 1
    elif(val == 0) :
      zeroes_index_list.append(j)
    j = j + 1

  # Modifier - the multiplier augments the number of zeroes passed into the data for slightly varied balancing
  num_ones = math.floor(num_ones*1.03)

  num_zeroes = num_instances - num_ones
  num_zeroes_to_remove = num_zeroes - num_ones

  # loop that removes "num_zeroes_to_remove" amount of zeroes
  for i in range(num_zeroes_to_remove) :
    potential_row = 0
    # Inner loop to prevent the same index getting selected for removal twice
    while True :

      index_matched = False
      potential_row = random.randint(0, num_zeroes-1)

      # Checking if the random index selected has been selected before
      for j in row_list:
        if (j == potential_row) :
          index_matched = True

      # If the random index has been selected before
      # Then you rerun the inner while loop
      # If not
      # Add the index to the index removal list and continue the for loop
      if (index_matched == False) :
        row_list.append(potential_row)
        return_index_list.append(zeroes_index_list[potential_row])
        break
  # Returns the balanced dataframe with the randomly selected zero indexes dropped
  return df.drop(index=return_index_list)

In [ ]:
# Running the undersampler function above
all_df_new = undersampler(all_df_new)
all_df_new.describe()

,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
count,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000,...,1.720000e+02,1.720000e+02,1.720000e+02,172.000000,1.720000e+02,172.000000,172.000000,172.000000,172.000000,172.000000
mean,0.488372,56.935078,51.931869,0.482558,6.174419,6.191860,6.145349,14.000000,7.436047,12.755814,...,3.983954e-01,3.937721e-01,2.700786e+00,0.003280,1.815656e+07,263.853946,34883.744128,0.058276,0.006188,0.020821
std,0.501324,26.518665,10.920595,0.501155,76.146472,76.145163,76.148645,107.151366,76.052394,107.286133,...,1.699934e-01,1.643529e-01,8.222805e-01,0.002427,5.722185e+07,1584.175457,184020.997341,0.048200,0.008611,0.063563
min,0.000000,9.000000,23.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,8.650000e-09,8.650000e-09,-3.200000e-16,0.000059,0.000000e+00,0.000000,0.000248,0.000000,0.000000,0.000000
25%,0.000000,38.000000,44.179329,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,...,3.275055e-01,3.275053e-01,2.251629e+00,0.001413,1.030473e+06,18.488987,0.001632,0.019493,0.000341,0.001294
50%,0.000000,55.000000,51.516633,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,...,4.077443e-01,4.042952e-01,2.778916e+00,0.002851,3.873320e+06,75.546660,0.003883,0.049721,0.003079,0.002825
75%,1.000000,73.812500,60.972792,1.000000,1.000000,1.000000,1.000000,3.000000,2.000000,1.000000,...,5.092063e-01,5.032208e-01,3.326590e+00,0.004601,9.430736e+06,170.971846,0.013578,0.086826,0.008499,0.009583
max,1.000000,144.000000,78.001369,1.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,...,8.652006e-01,8.571429e-01,4.947427e+00,0.011301,5.488188e+08,20764.693790,1000000.000000,0.285100,0.060742,0.518837


In [ ]:
all_df_new.info()

# Finds the mode of all of the values in the dataset while excluding missing values
def findModeExcludingMissing(df):
  mode = {}
  # Only looks at the first 12 columns as these are the only ones with missing values
  for col in df.columns[:12]:
    # Find mode excluding 999
    mode[col] = df[df[col] != 999][col].mode().values[0]
  return mode

# Fill in missing values with mode for that column
def replaceMissing(df, mode):
  df_temp = df
  for col in df_temp.columns[:12]:
    # Replace 999 with mode
    df_temp[col] = df_temp[col].replace(999, mode[col])
  return df_temp

# Running above functions and altering dataframe
mode = findModeExcludingMissing(all_df_new)
df_imputed = replaceMissing(all_df_new, mode)

df_imputed.describe()

temp_X = df_imputed.drop(['pCR (outcome)'], axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 394
Columns: 119 entries, pCR (outcome) to original_ngtdm_Strength
dtypes: float64(108), int64(11)
memory usage: 161.2 KB


In [ ]:
# Function to identify outliers using IQR
def outliers(df):
  index_list = []
  for col in df.columns[1:]:
    if df[col].dtype in [int, float]:

      # Retrieving values for upper and lower quartile and calculating Inter-Quartile Range
      Q1 = df[col].quantile(0.25)
      Q3 = df[col].quantile(0.75)
      IQR = Q3 - Q1

      # Calculating lower and upper bounds for outliers
      lower = Q1 - 1.5 * IQR
      upper = Q3 + 1.5 * IQR

      # Adding instance indexes that are above or below the bounds defined above to a list
      ls = df.index[(df[col] < lower) | (df[col] > upper)]
      index_list.extend(ls)

  # Returns a list of indexes to remove
  return ls

In [ ]:
index_list = outliers(df_imputed)

In [ ]:
# Function to remove rows with outliers
def remove(df, ls):
  ls = sorted(set(ls))
  df = df.drop(ls)
  return df

In [ ]:
df_cleaned = remove(df_imputed, index_list)

In [ ]:
df_cleaned.describe()

,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
count,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,...,1.470000e+02,1.470000e+02,1.470000e+02,147.000000,1.470000e+02,147.000000,147.000000,147.000000,147.000000,147.000000
mean,0.489796,57.102041,52.446059,0.469388,0.353741,0.374150,0.340136,2.435374,1.687075,1.156463,...,3.810371e-01,3.756277e-01,2.845024e+00,0.003397,1.443221e+07,307.917803,40816.331082,0.065918,0.007118,0.004030
std,0.501605,27.623782,10.801982,0.500768,0.479765,0.485557,0.475374,0.511083,0.817637,0.364535,...,1.625453e-01,1.549800e-01,7.790131e-01,0.002354,5.186986e+07,1710.522704,198540.944782,0.047162,0.008952,0.004361
min,0.000000,9.000000,29.670089,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,8.650000e-09,8.650000e-09,-3.200000e-16,0.000059,0.000000e+00,0.000000,0.000248,0.000000,0.000000,0.000000
25%,0.000000,38.000000,44.651609,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,...,3.221133e-01,3.221076e-01,2.428997e+00,0.001665,1.140288e+06,39.475236,0.001424,0.033156,0.001242,0.001175
50%,0.000000,53.000000,52.763860,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,...,3.993227e-01,3.976008e-01,3.009937e+00,0.002952,3.962034e+06,102.144179,0.002827,0.058387,0.003913,0.002333
75%,1.000000,74.000000,61.065024,1.000000,1.000000,1.000000,1.000000,3.000000,2.000000,1.000000,...,4.797157e-01,4.709722e-01,3.386186e+00,0.004660,8.969372e+06,188.010709,0.007050,0.095009,0.010815,0.005846
max,1.000000,144.000000,78.001369,1.000000,1.000000,1.000000,1.000000,3.000000,3.000000,2.000000,...,8.652006e-01,6.433012e-01,4.947427e+00,0.011301,5.488188e+08,20764.693790,1000000.000000,0.285100,0.060742,0.021448


In [ ]:
# Resetting indexes
num_instances = len(df_cleaned.index)

df_cleaned.index = [x for x in range(num_instances)]

# Defining our test features
X = df_cleaned.drop(['pCR (outcome)', 'RelapseFreeSurvival (outcome)'], axis=1)
X.info()

# Assuming 'pCR (outcome)' is target variable
y = df_cleaned['pCR (outcome)']
ys = pd.DataFrame(y)
ys.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147 entries, 0 to 146
Columns: 117 entries, Age to original_ngtdm_Strength
dtypes: float64(107), int64(10)
memory usage: 135.5 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 147 entries, 0 to 146
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   pCR (outcome)  147 non-null    int64
dtypes: int64(1)
memory usage: 2.3 KB


In [ ]:
#Using standard scaler as a form of data normalisation
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

Xs = scaler.fit_transform(X)

In [ ]:
from sklearn.decomposition import PCA

#Use PCA for dimensionality reduction on the normalized data
#This is to showcase and describe the data

# Number of principal components to calculate and reduce dimensionality down to
num_components = 40

# Defining, then Performing PCA
pca = PCA(n_components=num_components)

PCA_Xs = pca.fit_transform(Xs)

PCA_Xs_df = pd.DataFrame(PCA_Xs)
PCA_Xs_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
count,1.470000e+02,147.000000,1.470000e+02,147.000000,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02,...,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02,1.470000e+02
mean,-1.933450e-16,0.000000,-2.416812e-17,0.000000,4.833624e-17,-7.250436e-17,2.416812e-16,-6.042030e-18,2.416812e-16,-2.114711e-17,...,-1.510508e-17,-3.927320e-17,1.178196e-16,-1.510508e-17,4.909149e-17,3.021015e-18,-2.190236e-17,-1.963660e-17,3.304235e-17,7.099385e-17
std,5.987535e+00,4.264759,3.777409e+00,3.607293,2.333046e+00,2.082313e+00,1.858338e+00,1.690230e+00,1.523851e+00,1.457192e+00,...,4.386183e-01,4.039627e-01,3.980417e-01,3.574611e-01,3.305196e-01,2.978918e-01,2.861877e-01,2.697972e-01,2.645567e-01,2.531320e-01
min,-1.356563e+01,-12.463665,-5.682804e+00,-10.627770,-3.821287e+00,-7.715582e+00,-4.447819e+00,-6.042677e+00,-3.664390e+00,-4.846696e+00,...,-1.445804e+00,-1.041202e+00,-9.602340e-01,-8.314520e-01,-7.531869e-01,-7.238074e-01,-8.486838e-01,-7.038179e-01,-6.754300e-01,-6.383741e-01
25%,-4.440495e+00,-1.851094,-2.588788e+00,-1.976110,-1.550373e+00,-9.453591e-01,-1.071961e+00,-1.097169e+00,-9.339072e-01,-8.835844e-01,...,-2.475129e-01,-2.492284e-01,-2.698923e-01,-2.568616e-01,-2.179955e-01,-2.235073e-01,-1.624744e-01,-1.758995e-01,-1.651839e-01,-1.890718e-01
50%,-6.104024e-01,0.075978,-5.589934e-01,-0.029345,-4.776996e-01,3.733739e-01,-2.278449e-01,3.710259e-03,-1.074215e-01,-1.303295e-01,...,-7.071992e-02,-1.045017e-02,-4.903137e-04,-3.280206e-03,-1.930073e-02,7.383751e-04,-3.661574e-04,-8.251205e-03,-2.601838e-02,9.651937e-03
75%,3.791971e+00,1.690302,1.447136e+00,1.913455,1.080463e+00,1.226472e+00,7.516968e-01,1.147191e+00,1.067130e+00,7.502611e-01,...,2.596007e-01,2.817303e-01,2.502663e-01,2.157020e-01,1.952380e-01,1.932939e-01,1.709747e-01,1.599959e-01,1.314943e-01,1.560464e-01
max,2.322714e+01,16.796041,1.549775e+01,14.968212,1.368399e+01,7.769233e+00,8.400241e+00,6.452778e+00,5.225334e+00,7.748973e+00,...,1.534299e+00,1.115943e+00,1.235587e+00,1.519225e+00,8.528906e-01,8.951035e-01,9.375340e-01,8.253841e-01,7.456923e-01,1.192393e+00


In [ ]:
# Implementing test split with 60% of the variables being used for training
X_train, X_test, y_train, y_test = train_test_split(PCA_Xs_df, y, test_size=0.4, random_state=42)

In [ ]:
# Class for filling in missing values
# Created to be used in the pipeline
class valueFiller():
  def __init__(self, fitter, transformer):
    self.mode = {}
    self.fitter = fitter
    self.transformer = transformer

  # Saves the mode of the training data to ensure that most accurate modal value
  # is compiled with the test data
  def fit(self, X, y):
    self.mode = self.fitter(X)
    return self

  # Replaces all missing data with mode saved in the fit function
  def transform(self, X):
    return self.transformer(X, self.mode)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [ ]:
# Feature Selection using Wrapper Methods
# ANOVA
fs = SelectKBest(score_func=f_classif, k='all')
X_train_model = fs.fit_transform(X_train, y_train)
X_test_model = fs.transform(X_test)


# Train a SV classifier
svcC = 0.8
# Polynomial kernel
svcKernel = 'poly'
# Polynomial to the 9th degree
svcDegree = 9

clf = SVC(C=svcC, kernel=svcKernel,
          degree=svcDegree)
clf.fit(X_train_model, y_train)


classifier_score = clf.score(X_test_model, y_test)
print('The classification accuracy  is {:05.4f}'.format(classifier_score))

scores = cross_val_score(clf, X_test_model, y_test, cv=5)

print("Cross-validation scores: ",format(scores))
print("Mean accuracy: ", format(scores.mean()))
print("Standard deviation: ", format(scores.std()))

# Make predictions on the test data
y_pred = clf.predict(X_test_model)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

The classification accuracy  is 0.6271
Cross-validation scores:  [0.58333333 0.5        0.58333333 0.5        0.63636364]
Mean accuracy:  0.5606060606060607
Standard deviation:  0.05313841792954601
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.94      0.75        35
           1       0.67      0.17      0.27        24

    accuracy                           0.63        59
   macro avg       0.64      0.55      0.51        59
weighted avg       0.64      0.63      0.55        59



In [ ]:
# Defining valuefiller object before pipeline creation
po_valueFiller = valueFiller(fitter=findModeExcludingMissing, transformer=replaceMissing)

In [ ]:
# Train/Test split to do brief test of pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
# Making the pipeline
pipe = make_pipeline(
    po_valueFiller,
    StandardScaler(),
    PCA(n_components=num_components),
    SelectKBest(score_func=f_classif, k='all'),
    clf
)

pipe.fit(X, y)

Pipeline(steps=[('valuefiller',
                 <__main__.valueFiller object at 0x787d0e5f9f90>),
                ('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=40)),
                ('selectkbest', SelectKBest(k='all')),
                ('svc', SVC(C=0.8, degree=9, kernel='poly'))])

In [ ]:
# briefly testing pipeline
y_pred = pipe.predict(X_test)

scores = cross_val_score(pipe, X_test, y_test, cv=4)

print("Cross-validation scores: ",format(scores))
print("Mean accuracy: ", format(scores.mean()))
print("Standard deviation: ", format(scores.std()))

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Cross-validation scores:  [0.6        0.6        0.53333333 0.64285714]
Mean accuracy:  0.594047619047619
Standard deviation:  0.03917733969501033
Classification Report:
               precision    recall  f1-score   support

           0       0.67      1.00      0.80        35
           1       1.00      0.29      0.45        24

    accuracy                           0.71        59
   macro avg       0.84      0.65      0.63        59
weighted avg       0.81      0.71      0.66        59



In [ ]:
import pickle

# Pickling pipeline for use on test set
pickle.dump(pipe, open('classification_model.pkl','wb'))